In [1]:
import cv2
from tqdm import tqdm
import os
import glob
import numpy as np

In [2]:
frames_dir = 'preprocessed-frames'
output_dir = 'motion-detection-frames'

In [3]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
out = cv2.VideoWriter("motion-detection-output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 15, (3420,1910))


In [5]:
for i in tqdm(range(len(glob.glob1(frames_dir, "*.jpg")) - 1)):
    prev_frame = cv2.imread(f"{frames_dir}/frame{i}.jpg")
    curr_frame = cv2.imread(f"{frames_dir}/frame{i + 1}.jpg")

    diff = cv2.absdiff(curr_frame, prev_frame)
    _, binary_diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    clean_diff = cv2.morphologyEx(binary_diff, cv2.MORPH_CLOSE, kernel)
    
    out.write(clean_diff)
    cv2.imwrite(f"{output_dir}/frame{i}.jpg", clean_diff)    

    # flow = cv2.calcOpticalFlowFarneback(prev_frame, curr_frame, None,
    #                                 0.5, 3, 15, 3, 5, 1.2, 0)
    # magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
out.release()

100%|██████████| 749/749 [03:04<00:00,  4.05it/s]


MOTION DETECTION USING Motion-Compensated Filtering

In [6]:
output_dir = 'motion-detection-frames-MCF'

In [7]:


for i in tqdm(range(len(glob.glob1(frames_dir, "*.jpg")) - 1)):
    prev_frame = cv2.imread(f"{frames_dir}/frame{i}.jpg")
    #prev_frame.size
    curr_frame = cv2.imread(f"{frames_dir}/frame{i + 1}.jpg")
    #curr_frame.size
    
    # Calcolo del flusso ottico (metodo di Farneback)
    flow = cv2.calcOpticalFlowFarneback(prev_frame, curr_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Creazione della mappa del movimento compensata
    h, w = prev_frame.shape
    compensated = np.zeros_like(prev_frame, dtype=np.float32)

    for y in range(h):
        for x in range(w):
            dx, dy = flow[y, x]
            x_comp = int(x + dx)
            y_comp = int(y + dy)

            # Controlla che il pixel rientri nell'immagine
            if 0 <= x_comp < w and 0 <= y_comp < h:
                compensated[y, x] = prev_frame[y_comp, x_comp]

    # Filtro temporale lungo la traiettoria
    #filtered = cv2.GaussianBlur(compensated, (5, 5), 0)

    # Differenza tra frame originale e compensato
    #motion_map = cv2.absdiff(prev_frame, filtered)
    motion_map = cv2.absdiff(prev_frame, compensated)
    _, binary_motion_map = cv2.threshold(motion_map, 25, 255, cv2.THRESH_BINARY)
    out.write(binary_motion_map)
    cv2.imwrite(f"{output_dir}/frame{i}.jpg", binary_motion_map)    
    
out.release()

  0%|          | 0/749 [00:00<?, ?it/s]


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\video\src\optflowgf.cpp:1116: error: (-215:Assertion failed) prev0.size() == next0.size() && prev0.channels() == next0.channels() && prev0.channels() == 1 && pyrScale_ < 1 in function 'cv::`anonymous-namespace'::FarnebackOpticalFlowImpl::calc'
